In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
cifar10 = keras.datasets.cifar10
(X_train_full, y_train_full),(X_test,y_test) = cifar10.load_data()

In [ ]:
X_train_full.shape

In [ ]:
X_test.shape

In [ ]:
X_train_full[0]

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(10,10))
a = 0

for i in range(3):
    for j in range(3):        
        axes[i, j].imshow(X_train_full[a], cmap=plt.get_cmap('gray'))
        a = a + 1

plt.show()

In [ ]:
class_names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [ ]:
class_names[y_train_full[8][0]]

In [ ]:
plt.imshow(X_train_full[8], cmap=plt.get_cmap('gray'))

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255, X_train_full[5000:]/255
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test/255

In [ ]:
X_train[0]

In [ ]:
# # 1cycle learning rate scheduler
# # code from: https://pub.towardsai.net/1cycle-learning-rate-scheduling-with-tensorflow-and-keras-74ae7ce277c8
# class OneCycleScheduler(keras.callbacks.Callback):
#     def __init__(self, iterations, max_rate, start_rate=None,
#                  last_iterations=None, last_rate=None):
#         self.iterations = iterations
#         self.max_rate = max_rate
#         self.start_rate = start_rate or max_rate / 10
#         self.last_iterations = last_iterations or iterations // 10 + 1
#         self.half_iteration = (iterations - self.last_iterations) // 2
#         self.last_rate = last_rate or self.start_rate / 1000
#         self.iteration = 0
        
#     def _interpolate(self, iter1, iter2, rate1, rate2):
#         return ((rate2 - rate1) * (self.iteration - iter1)
#                 / (iter2 - iter1) + rate1)
    
#     def on_batch_begin(self, batch, logs):
#         if self.iteration < self.half_iteration:
#             rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
#         elif self.iteration < 2 * self.half_iteration:
#             rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
#                                      self.max_rate, self.start_rate)
#         else:
#             rate = self._interpolate(2 * self.half_iteration, self.iterations,
#                                      self.start_rate, self.last_rate)
#         self.iteration += 1
#         keras.backend.set_value(self.model.optimizer.lr, rate)
        

# onecycle = OneCycleScheduler(30,0.04)
# , callbacks=[onecycle]

In [ ]:
def exponential_decay(epoch):
        return 0.01*0.1**(epoch/10)
    
exponential_decay_fn = exponential_decay
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
model = keras.models.Sequential([
            keras.layers.Flatten(input_shape=[32,32,3]),
            keras.layers.AlphaDropout(rate=0.1),
            keras.layers.Dense(300, kernel_initializer='lecun_normal', activation='selu'),
            keras.layers.AlphaDropout(rate=0.1),
            keras.layers.Dense(300, kernel_initializer='lecun_normal', activation='selu'),
            keras.layers.AlphaDropout(rate=0.1),
            keras.layers.Dense(300, kernel_initializer='lecun_normal', activation='selu'),
            keras.layers.AlphaDropout(rate=0.1),
            keras.layers.Dense(300, kernel_initializer='lecun_normal', activation='selu'),
            keras.layers.AlphaDropout(rate=0.1),
            keras.layers.Dense(300, kernel_initializer='lecun_normal', activation='selu'),
            keras.layers.AlphaDropout(rate=0.1),
            keras.layers.Dense(10, activation = "softmax")
            ])

In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=60, validation_data = (X_valid, y_valid), batch_size=32, callbacks=[lr_scheduler])


In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(15,8))
plt.grid(True)
plt.gca()
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_prob = model.predict(X_test)
y_classes = y_prob.argmax(axis=-1)
y_classes

In [ ]:
confusion_matrix = tf.math.confusion_matrix(y_test, y_classes)

In [ ]:
import seaborn as sb    

# ax = plt.figure(figsize=(8, 6))
fig = sb.heatmap(confusion_matrix, annot=True, fmt='g', cmap='Greens')  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
fig.set_xlabel('Predicted labels')
fig.set_ylabel('True labels')
fig.set_title('Confusion Matrix')
fig.xaxis.set_ticklabels(class_names) 
fig.yaxis.set_ticklabels(class_names)
fig.figure.set_size_inches(10, 10)


plt.show()